In [1]:
import os
import shutil
import re
from concurrent.futures import ThreadPoolExecutor

In [2]:
image_dir = "/data/glusterfs/agurianova/rolling-in-the-deep/data/deepvariant/images200k"

In [32]:
max_files = 100000

# Gather the first 1000 eligible files
files_to_process = []
with os.scandir(image_dir) as entries:
    for entry in entries:
        if entry.is_file() and entry.name.endswith(".png"):
            files_to_process.append(entry.path)
            if len(files_to_process) >= max_files:
                break

def move_file(src_path):
    file_name = os.path.basename(src_path)
    print(f"Processing: {file_name}")
    match = re.search(r'_label([012])_', file_name)
    if match:
        label = match.group(1)
        dst_path = os.path.join(image_dir, label, file_name)
        shutil.move(src_path, dst_path)
    else:
        print(f"Label not found for: {file_name}")

# Process files in parallel using threads
with ThreadPoolExecutor() as executor:
    executor.map(move_file, files_to_process)